### Welcome to mathématical planning.




# Techno used :

Mainly based on the llm through the libraries langchain, langgraph and langsmith.
At first we'll use a notebook to configure and test the design of the differents parts of the workflow.
Next, we'll build the workflow mith langgraph
Then, we'll use "deep_note" and streamlit to make the final tool

## Load the API keys

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

In [3]:
from tools.job_and_resume import create_vectorstore, question_to_vectorstore

# 0- Init the flow

First I will work with the most classic openai, gpt-4o-mini. I love the ollama deepseek-r1 mais il est si lent à tourner en local.
Peut être à faire évoluer vers du cloud.



In [4]:
# Installation des dépendances nécessaires
# pip install langchain langchain-community pypdf

# Chemin vers votre fichier PDF
pdf_path = "./data_test/proba.pdf"


# 1- load the sources pdf 

## 1-1 Load the resume

In [5]:
if not os.path.exists(pdf_path):
    print(f"Erreur: Le fichier {pdf_path} n'existe pas.")

    
def demo_interactive():
    """
    Démonstration interactive d'interrogation de la base de connaissances
    """
    # Création de la base de connaissances
    vectorstore = create_vectorstore(pdf_path)
    
    print("\n=== Assistant de cours de probabilités ===")
    print("Posez des questions sur votre cours (ou 'q' pour quitter)")
    

    question = "Peux tu extraire la table des matières ?"
        
    reponse, docs = question_to_vectorstore(vectorstore, question)
        
    print("\nRéponse:")
    print(reponse)
        
    print("\nSources (premiers mots):")
    for i, doc in enumerate(docs):
            # Affiche les premiers mots de chaque source
        preview = doc.page_content[:100].replace('\n', ' ') + "..."
        print(f"Document {i+1} (Page {doc.metadata['page']}): {preview}")

    return reponse

table_matiere = demo_interactive()
    


Extraction du contenu du PDF: ./data_test/proba.pdf
Nombre de pages extraites: 31
Nombre de chunks créés: 34
Base de connaissances créée avec 34 chunks

=== Assistant de cours de probabilités ===
Posez des questions sur votre cours (ou 'q' pour quitter)


c:\Users\aufol\OneDrive\Bureau\projet_langchain\cours_planning\tools\job_and_resume.py:79: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")
c:\Users\aufol\OneDrive\Bureau\projet_langchain\cours_planning\tools\job_and_resume.py:82: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  reponse = chain.run(input_documen


Réponse:
Voici la table des matières extraite :

1. Dénombrement
   1.1 Permutations
   1.2 Arrangements
   1.3 Combinaisons
   1.4 Principes additif et multiplicatif
2. Introduction aux probabilités
   2.1 Généralités
   2.2 Axiomes
   2.3 Equiprobabilité
3. Probabilités conditionnelles
4. Variables aléatoires discrètes
   4.1 Introduction et premier exemple
   4.2 Définitions et propriétés générales
   4.3 Lois notables
      4.3.1 Loi équiprobable
      4.3.2 Loi binomiale
      4.3.3 Loi hypergéométrique
      4.3.4 Loi géométrique
5. Variables aléatoires continues
   5.1 Définitions et propriétés générales
   5.2 Lois notables
      5.2.1 Loi uniforme
      5.2.2 Loi Normale
6. Propriétés de la fonction de densité
7. La propriété des « 3 écarts-types »

Sources (premiers mots):
Document 1 (Page 4): Exemple11. OUntouristeperdupartdupointOetfait5passurl'axe(1pas=1unité)verslagaucheouversladroite.Com...
Document 2 (Page 2): Exemple6.Combiend'équipesde3collèguespeut-onformerdansuneen

In [6]:
from flow_tools.nodes import model_query
from langchain_openai import ChatOpenAI

question_template = """Peux tu extraire les titres principaux de la table de matières suivante : 
    {table_matiere}
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

questions_to_the_candidate = model_query(llm=llm, prompt_template=question_template, inputs={"table_matiere":table_matiere})
    

In [7]:
print(questions_to_the_candidate)

Voici les titres principaux de la table des matières que vous avez fournie :

1. Dénombrement
2. Introduction aux probabilités
3. Probabilités conditionnelles
4. Variables aléatoires discrètes
5. Variables aléatoires continues
6. Propriétés de la fonction de densité
7. La propriété des « 3 écarts-types »


Voici les titres principaux extraits de la table des matières :

1. Dénombrement
2. Introduction aux probabilités
3. Probabilités conditionnelles
4. Variables aléatoires discrètes
5. Variables aléatoires continues

In [8]:
# Extraction des définitions et exercices du chapitre dénombrement
from tools.job_and_resume import extract_content_pdf

pdf_content = "".join([t.page_content for t in extract_content_pdf(pdf_path)])



Extraction du contenu du PDF: ./data_test/proba.pdf
Nombre de pages extraites: 31


In [9]:
len(pdf_content)

43213

### Extraction des data du premier chapitre

In [10]:
nom_chapitre = "1. Dénombrement"
question_template = """Peux tu extraire le chapitre {nom_chapitre} du texte suivant : 
    {pdf_content}
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chapitre = model_query(llm=llm, prompt_template=question_template, inputs={"nom_chapitre":nom_chapitre, "pdf_content":pdf_content})

In [11]:
print(chapitre)

Voici l'extraction du chapitre 1 "Dénombrement" du texte fourni :

---

# 1 DÉNOMBREMENT

## 1.1 PERMUTATIONS

**DÉFINITION 1.**  
Soit n un entier positif ou nul. Le nombre de permutations de n éléments distincts est le nombre de manières de les ordonner. On le note P_n et on a P_n = n! = 1 × 2 × ⋯ × (n−2) × (n−1) × n

**Exemple 2.**  
Combien d'anagrammes du mot COMPTE peut-on former? Combien de classements sont possibles dans une compétition entre huit athlètes (sans ex-aequo)?

## 1.2 ARRANGEMENTS

**DÉFINITION 3.**  
Soit deux entiers 0 ≤ k ≤ n. Le nombre d'arrangements de k éléments pris parmi n éléments distincts est le nombre de manières de choisir et d'ordonner ces k éléments. On le note A_n,k et on a A_n,k = n! / (n−k)!

**Exemple 4.**  
Combien de podiums (or, argent, bronze) peut-on former dans le cadre d'une compétition olympique entre 8 athlètes? J'ai 7 tâches à accomplir aujourd'hui. Combien de manières aurai-je de constituer une liste de 4 tâches prioritaires, classées 

In [12]:
nom_chapitre = "1. Dénombrement"
question_template = """Peux tu extraire les définitions du chapitre suivant : 
    {chapitre}
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

definitions = model_query(llm=llm, prompt_template=question_template, inputs={"chapitre":chapitre})

In [13]:
print(definitions)

Voici les définitions extraites du chapitre 1 "Dénombrement" :

---

## 1.1 PERMUTATIONS

**DÉFINITION 1.**  
Soit n un entier positif ou nul. Le nombre de permutations de n éléments distincts est le nombre de manières de les ordonner. On le note P_n et on a P_n = n! = 1 × 2 × ⋯ × (n−2) × (n−1) × n.

---

## 1.2 ARRANGEMENTS

**DÉFINITION 3.**  
Soit deux entiers 0 ≤ k ≤ n. Le nombre d'arrangements de k éléments pris parmi n éléments distincts est le nombre de manières de choisir et d'ordonner ces k éléments. On le note A_n,k et on a A_n,k = n! / (n−k)!.

---

## 1.3 COMBINAISONS

**DÉFINITION 5.**  
Soit deux entiers 0 ≤ k ≤ n. Le nombre de combinaisons de k éléments pris parmi n est le nombre de manières de choisir ces k éléments sans les ordonner. On le note C(n, k) et on a C(n, k) = n! / (k!(n−k)!).

---

## 1.4 PRINCIPES ADDITIF ET MULTIPLICATIF

**PROPOSITION 10.**  
On suppose qu'il y a N1 TRUCS et N2 MACHINS. On suppose aussi qu'un BIDULE peut être soit un TRUC soit un MACHIN m

In [14]:
question_template = """Peux tu commenter et illustrer chacune des définitions : 
    {definitions}
"""

definitions_illustrated = model_query(llm=llm, prompt_template=question_template, inputs={"definitions":definitions})

In [15]:
print(definitions_illustrated)

Bien sûr, je vais commenter et illustrer chacune des définitions que vous avez fournies.

---

## 1.1 PERMUTATIONS

**DÉFINITION 1.**  
Soit n un entier positif ou nul. Le nombre de permutations de n éléments distincts est le nombre de manières de les ordonner. On le note P_n et on a P_n = n! = 1 × 2 × ⋯ × (n−2) × (n−1) × n.

### Commentaire :
Les permutations concernent l'arrangement de tous les éléments d'un ensemble. Par exemple, si nous avons trois éléments distincts A, B et C, les permutations possibles sont ABC, ACB, BAC, BCA, CAB et CBA. Il y a donc 6 façons d'ordonner ces trois éléments, ce qui correspond à 3! = 6.

### Illustration :
Pour n = 3 (A, B, C) :
- ABC
- ACB
- BAC
- BCA
- CAB
- CBA

---

## 1.2 ARRANGEMENTS

**DÉFINITION 3.**  
Soit deux entiers 0 ≤ k ≤ n. Le nombre d'arrangements de k éléments pris parmi n éléments distincts est le nombre de manières de choisir et d'ordonner ces k éléments. On le note A_n,k et on a A_n,k = n! / (n−k)!.

### Commentaire :
Les arrange

In [16]:
question_template = """Peux tu compléter chacune de ces définitions, complétant l'illustration, par un exercice classique dont tu donnes le résultat, pouis le résultat avec un corrigé : 
    {definitions_illustrated}
"""

definitions_exercice = model_query(llm=llm, prompt_template=question_template, inputs={"definitions_illustrated":definitions_illustrated})

In [17]:
print(definitions_exercice)

Bien sûr, je vais compléter chaque définition avec un exercice classique, donner le résultat, puis fournir le corrigé.

---

## 1.1 PERMUTATIONS

**DÉFINITION 1.**  
Soit n un entier positif ou nul. Le nombre de permutations de n éléments distincts est le nombre de manières de les ordonner. On le note P_n et on a P_n = n! = 1 × 2 × ⋯ × (n−2) × (n−1) × n.

### Exercice :
Calculez le nombre de permutations de 4 éléments distincts (A, B, C, D).

### Résultat :
P_4 = 4! = 24.

### Corrigé :
Pour calculer P_4, nous utilisons la définition :
\[ P_4 = 4! = 4 × 3 × 2 × 1 = 24. \]
Il y a donc 24 façons d'ordonner les 4 éléments A, B, C et D.

---

## 1.2 ARRANGEMENTS

**DÉFINITION 3.**  
Soit deux entiers 0 ≤ k ≤ n. Le nombre d'arrangements de k éléments pris parmi n éléments distincts est le nombre de manières de choisir et d'ordonner ces k éléments. On le note A_n,k et on a A_n,k = n! / (n−k)!.

### Exercice :
Calculez le nombre d'arrangements de 3 éléments pris parmi 5 éléments distincts (A,

In [18]:
import pdfplumber

def extract_text_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Utilisation
pdf_text = extract_text_pdfplumber(pdf_path)
print(pdf_text)


Table des matières
1 Dénombrement . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
1.1 Permutations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
1.2 Arrangements . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
1.3 Combinaisons . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
1.4 Principes additif et multiplicatif . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
2 Introduction aux probabilités . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7
2.1 Généralités . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8
2.2 Axiomes . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10
2.3 Equiprobabilité . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 11
3 Probabilités conditionnelles . . . . . . . . . . . . . . . . . . . . . . .

In [19]:
nom_chapitre = "1. Dénombrement"
question_template = """Peux tu extraire le chapitre {nom_chapitre} du texte suivant : 
    {pdf_content}
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chapitre = model_query(llm=llm, prompt_template=question_template, inputs={"nom_chapitre":nom_chapitre, "pdf_content":pdf_text})

In [20]:
question_template = """Peux tu extraire les définitions du chapitre suivant : 
    {chapitre}
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

definitions = model_query(llm=llm, prompt_template=question_template, inputs={"chapitre":chapitre})

In [21]:
print(definitions)

Voici les définitions extraites du chapitre 1 "Dénombrement" :

### 1.1 PERMUTATIONS

**DÉFINITION 1.**  
Le nombre de permutations de n éléments distincts est le nombre de manières de les ordonner. On le note P et on a  
\[ P_n = n! = 1 \times 2 \times \cdots \times (n-2) \times (n-1) \times n \]

### 1.2 ARRANGEMENTS

**DÉFINITION 3.**  
Le nombre d'arrangements de k éléments pris parmi n éléments distincts est le nombre de manières de choisir et d'ordonner ces k éléments. On le note \( A_k \) et on a  
\[ A_k = \frac{n!}{(n-k)!} \]

### 1.3 COMBINAISONS

**DÉFINITION 5.**  
Le nombre de combinaisons de k éléments parmi n est le nombre de manières de choisir ces k éléments sans les ordonner. On le note \( \binom{n}{k} \) et on a  
\[ \binom{n}{k} = \frac{n!}{k!(n-k)!} \]

### 1.4 PRINCIPES ADDITIF ET MULTIPLICATIF

**PROPOSITION 10.**  
On peut conclure qu'il y a \( N_1 + N_2 \) BIDULES si un BIDULE peut être soit un TRUC soit un MACHIN mais pas les deux à la fois.

**PROPOSITION 12.

In [22]:
question_template = """Peux tu commenter et illustrer chacune des définitions, puis mettre en forme le résultat pour un lecteur humain : 
    {definitions}
"""

definitions_illustrated = model_query(llm=llm, prompt_template=question_template, inputs={"definitions":definitions})

In [23]:
print(definitions_illustrated)

Voici une présentation commentée et illustrée des définitions extraites du chapitre 1 "Dénombrement". Chaque concept est expliqué avec des exemples pour faciliter la compréhension.

---

## 1.1 PERMUTATIONS

**DÉFINITION 1.**  
Le nombre de permutations de \( n \) éléments distincts est le nombre de manières de les ordonner. On le note \( P_n \) et on a  
\[ P_n = n! = 1 \times 2 \times \cdots \times (n-2) \times (n-1) \times n \]

### Commentaire :
Les permutations concernent l'arrangement de tous les éléments d'un ensemble. Par exemple, si nous avons 3 éléments distincts : A, B, et C, les permutations possibles sont : ABC, ACB, BAC, BCA, CAB, CBA. 

### Illustration :
Pour \( n = 3 \) (A, B, C) :
- \( P_3 = 3! = 3 \times 2 \times 1 = 6 \) permutations.

---

## 1.2 ARRANGEMENTS

**DÉFINITION 3.**  
Le nombre d'arrangements de \( k \) éléments pris parmi \( n \) éléments distincts est le nombre de manières de choisir et d'ordonner ces \( k \) éléments. On le note \( A_k \) et on a  
\

In [24]:
question_template = """Peux tu m'expliquer et illustrer ce qu'est un arrangement, puis propose un exercice en t'inspirant du chapitre suivant : 
    {chapitre}
"""

question_about = model_query(llm=llm, prompt_template=question_template, inputs={"chapitre":chapitre})

print(question_about)

## Qu'est-ce qu'un arrangement ?

Un arrangement est une manière de choisir et d'ordonner un certain nombre d'éléments à partir d'un ensemble donné. Contrairement aux combinaisons, où l'ordre n'a pas d'importance, dans les arrangements, l'ordre des éléments est crucial. 

### Définition

Soit \( n \) le nombre total d'éléments distincts et \( k \) le nombre d'éléments à choisir et à ordonner. Le nombre d'arrangements de \( k \) éléments parmi \( n \) est donné par la formule :

\[
A_k = \frac{n!}{(n-k)!}
\]

où \( n! \) (factorielle de \( n \)) est le produit de tous les entiers de 1 à \( n \).

### Illustration

Prenons un exemple simple : imaginons que nous avons 5 livres distincts : A, B, C, D, et E. Si nous voulons savoir de combien de façons nous pouvons choisir et ordonner 3 de ces livres, nous utiliserons la formule des arrangements.

1. **Calculons \( A_3 \) pour \( n = 5 \)** :
   \[
   A_3 = \frac{5!}{(5-3)!} = \frac{5!}{2!} = \frac{5 \times 4 \times 3 \times 2 \times 1}{2 \t

In [25]:
theme="Combinaison"

question_template = """Peux tu m'expliquer et illustrer ce qu'est un {theme}, puis propose un exercice en t'inspirant du chapitre suivant : 
    {chapitre}
"""

question_about = model_query(llm=llm, prompt_template=question_template, inputs={"theme":theme, "chapitre":chapitre})

print(question_about)

## Combinaisons

### Définition
Une combinaison est un moyen de sélectionner un sous-ensemble d'éléments à partir d'un ensemble plus grand, sans tenir compte de l'ordre. Par exemple, si vous avez un ensemble de 5 fruits : {pomme, banane, orange, raisin, poire}, et que vous souhaitez choisir 2 fruits, les combinaisons possibles seraient {pomme, banane}, {pomme, orange}, {pomme, raisin}, {pomme, poire}, {banane, orange}, {banane, raisin}, {banane, poire}, {orange, raisin}, {orange, poire}, et {raisin, poire}. 

### Notation
Le nombre de combinaisons de \( k \) éléments parmi \( n \) éléments distincts est noté \( \binom{n}{k} \) et est calculé par la formule :
\[
\binom{n}{k} = \frac{n!}{k!(n-k)!}
\]

### Exemple
Si vous avez 10 employés dans une entreprise et que vous souhaitez former une équipe de 3 personnes, le nombre de façons de choisir ces 3 employés sans tenir compte de l'ordre est donné par :
\[
\binom{10}{3} = \frac{10!}{3!(10-3)!} = \frac{10 \times 9 \times 8}{3 \times 2 \time

In [26]:
theme="Principes additifs et multiplicatifs"

question_template = """Peux tu m'expliquer et illustrer ce qu'est un {theme}, puis propose un exercice en t'inspirant du chapitre suivant : 
    {chapitre}
"""

question_about = model_query(llm=llm, prompt_template=question_template, inputs={"theme":theme, "chapitre":chapitre})

print(question_about)

Les principes additifs et multiplicatifs sont des concepts fondamentaux en combinatoire qui permettent de calculer le nombre total de façons d'effectuer des choix ou des arrangements.

### Principe Additif

Le principe additif s'applique lorsque l'on a plusieurs options qui ne se chevauchent pas. Si une situation peut se produire de plusieurs manières distinctes, le nombre total de manières de réaliser cette situation est la somme des manières de chaque option.

**Exemple :**  
Supposons que vous ayez 3 chemises (rouge, bleue, verte) et 2 pantalons (noir, beige). Si vous voulez savoir combien de tenues différentes vous pouvez porter, vous pouvez choisir une chemise et un pantalon. Le nombre total de tenues est donné par :

- Tenues avec chemise rouge : 2 (noir, beige)
- Tenues avec chemise bleue : 2 (noir, beige)
- Tenues avec chemise verte : 2 (noir, beige)

Donc, le nombre total de tenues est \( 2 + 2 + 2 = 6 \).

### Principe Multiplicatif

Le principe multiplicatif s'applique lorsq

In [27]:
theme="Permutations"

question_template = """Peux tu m'expliquer et illustrer ce qu'est un {theme}, puis propose un exercice en t'inspirant du chapitre suivant : 
    {chapitre}
"""

question_about = model_query(llm=llm, prompt_template=question_template, inputs={"theme":theme, "chapitre":chapitre})

print(question_about)

## Explication des Permutations

Les **permutations** sont des arrangements d'un ensemble d'éléments distincts. Lorsque nous parlons de permutations, nous nous intéressons à la manière dont nous pouvons ordonner ces éléments. Par exemple, si nous avons trois lettres A, B et C, les différentes permutations de ces lettres sont :

- ABC
- ACB
- BAC
- BCA
- CAB
- CBA

Pour un ensemble de \( n \) éléments distincts, le nombre total de permutations est donné par \( n! \) (n factoriel), qui est le produit de tous les entiers de 1 à \( n \). Par exemple, pour \( n = 3 \) :

\[
P_3 = 3! = 3 \times 2 \times 1 = 6
\]

### Illustration

Prenons un exemple concret : considérons le mot "CHAT". Les lettres de ce mot sont distinctes, et nous voulons savoir combien de façons nous pouvons les arranger. 

Le nombre de permutations de "CHAT" est :

\[
P_4 = 4! = 4 \times 3 \times 2 \times 1 = 24
\]

Cela signifie qu'il existe 24 anagrammes différentes que l'on peut former avec les lettres de "CHAT".

## E

In [31]:
from langchain_community.document_loaders.image import UnstructuredImageLoader

loader = UnstructuredImageLoader("./data_test/question_qcm.png")

data = loader.load()

data[0]

ModuleNotFoundError: No module named 'pi_heif'

In [28]:


question_template = """Peux tu me citer des vers de shakespeare ? 
    
"""

question_about = model_query(llm=llm, prompt_template=question_template, inputs={})

print(question_about)

Bien sûr ! Voici quelques vers célèbres de William Shakespeare :

1. **Roméo et Juliette** :
   "Mais, doux amour, je t'aime, et je suis à toi."

2. **Hamlet** :
   "Être ou ne pas être, telle est la question."

3. **Le Songe d'une nuit d'été** :
   "L'amour ne voit pas avec les yeux, mais avec l'âme."

4. **Othello** :
   "Je suis un homme, et rien de ce qui est humain ne m'est étranger."

5. **La Tempête** :
   "Nous sommes tels des rêves, et nos vies ne sont que des ombres."

Si tu souhaites des citations sur un thème particulier ou d'une œuvre spécifique, fais-le moi savoir !
